# Prepare Dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Example: Loading data (modify according to actual dataset structure)
# Assuming images are stored in a directory structure: dataset/class_label/image.png

import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define dataset path
dataset_path = 'path_to_your_dataset'

# Parameters
img_height, img_width = 64, 64  # Resize images if necessary
batch_size = 32

# Use ImageDataGenerator for loading and preprocessing
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.4)  # 60% training, 20% val, 20% test

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Split validation into validation and test (each 20%)
val_data, test_data = train_test_split(validation_generator, test_size=0.5, random_state=42)


# Split the Dataset

In [ ]:
# Example using numpy arrays (if applicable)

# Assuming X and y are your data and labels
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# One-hot encode labels if necessary
y_train = to_categorical(y_train, num_classes=K)
y_val = to_categorical(y_val, num_classes=K)
y_test = to_categorical(y_test, num_classes=K)


# Build the CNN Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define model parameters
x1 = 32       # Number of filters for first Conv layer
m1 = 3        # Kernel size for first Conv layer
x2 = 64       # Number of filters for second Conv layer
m2 = 3        # Kernel size for second Conv layer
x3 = 128      # Number of units in Fully Connected layer
d = 0.5       # Dropout rate
K = train_generator.num_classes  # Number of classes

model = Sequential([
    Conv2D(x1, (m1, m1), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(x2, (m2, m2), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(x3, activation='relu'),
    Dropout(d),
    Dense(K, activation='softmax')
])

model.summary()


# Train the Model

In [ ]:
from tensorflow.keras.optimizers import Adam

# Compile the model
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',  # Use 'sparse_categorical_crossentropy' if labels are integers
              metrics=['accuracy'])

# Train the model
epochs = 20

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

# Plot training and validation loss
plt.figure(figsize=(12, 4))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss vs. Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Accuracy plot
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy vs. Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


# Evaluate the Model

## Evaluate on Test Data:

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {test_accuracy*100:.2f}%")


## Generate Predictions:

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Predict classes
y_pred = model.predict(test_data)
y_pred_classes = np.argmax(y_pred, axis=1)

# True classes
y_true = test_data.classes

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred_classes)
print("Confusion Matrix:")
print(cm)

# Classification Report
cr = classification_report(y_true, y_pred_classes, target_names=test_data.class_indices.keys())
print("Classification Report:")
print(cr)


## Plot Confusion Matrix:

In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=test_data.class_indices.keys(),
            yticklabels=test_data.class_indices.keys())
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()


# Plot Training and Validation Loss for Different Learning Rates

In [ ]:
learning_rates = [0.0001, 0.001, 0.01, 0.1]
history_dict = {}

for lr in learning_rates:
    print(f"\nTraining with learning rate: {lr}")
    optimizer = Adam(learning_rate=lr)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    history = model.fit(
        train_generator,
        epochs=20,
        validation_data=validation_generator,
        verbose=0  # Suppress output for brevity
    )
    history_dict[lr] = history.history

# Plotting
plt.figure(figsize=(15, 10))
for lr, history in history_dict.items():
    plt.plot(history['val_loss'], label=f'Validation Loss (lr={lr})')
    plt.plot(history['loss'], label=f'Training Loss (lr={lr})', linestyle='--')

plt.title('Training and Validation Loss for Different Learning Rates')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()
